In [1]:
import pandas as pd
import numpy as np
import requests
from tmdbv3api import TMDb, Movie
import bs4 as bs
import urllib.request

## Extract movie details for year 2016 from wikipedia page

In [2]:
source = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_American_films_of_2016").read()
soup = bs.BeautifulSoup(source, 'lxml')
tables = soup.find_all('table', class_='wikitable sortable')

In [3]:
# Extracting features from Wikipedia
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]))[0]

In [4]:
df_2016 = df1.append([df2,df3,df4], ignore_index=True)
df_2016

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\1535520579.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2016 = df1.append([df2,df3,df4], ignore_index=True)


,Opening,Opening.1,Title,Production Company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Production company,Ref.
0,JANUARY,8.0,The Forest,Gramercy Pictures / Focus Features,"Jason Zada (director); Ben Ketai, Sarah Cornwe...",[2],NaN,NaN
1,JANUARY,8.0,Anesthesia,IFC Films,Tim Blake Nelson (director/screenplay); Glenn ...,[3],NaN,NaN
2,JANUARY,8.0,Lamb,The Orchard,Ross Partridge (director/screenplay); Ross Par...,[4],NaN,NaN
3,JANUARY,15.0,Ride Along 2,Universal Pictures,"Tim Story (director); Phil Hay, Matt Manfredi ...",[5],NaN,NaN
4,JANUARY,15.0,13 Hours: The Secret Soldiers of Benghazi,Paramount Pictures,Michael Bay (director); Chuck Hogan (screenpla...,[6],NaN,NaN
...,...,...,...,...,...,...,...,...
173,DECEMBER,23.0,Silence,NaN,Martin Scorsese (director/screenplay); Jay Coc...,NaN,Paramount Pictures,[173]
174,DECEMBER,23.0,Why Him?,NaN,John Hamburg (director/screenplay); Ian Helfer...,NaN,20th Century Fox / 21 Laps Entertainment / Red...,[174]
175,DECEMBER,25.0,Hidden Figures,NaN,Theodore Melfi (director/screenplay); Allison ...,NaN,20th Century Fox / Chernin Entertainment,[175]
176,DECEMBER,25.0,Live by Night,NaN,Ben Affleck (director/screenplay); Ben Affleck...,NaN,Warner Bros. Pictures / RatPac Entertainment,[176]


In [5]:
df_2016 = df_2016[['Title', 'Cast and crew']]

In [6]:
df_2016['year'] = 2016.0

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\3968155689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['year'] = 2016.0


In [7]:
# Initialize TMDB API
tmdb = TMDb()
tmdb.api_key = 'Your TMDB Key'

tmdb_movie = Movie()

In [8]:
def get_movie_details(x):
    result = tmdb_movie.search(x)
    if result:
        movie = result[0]
        movie_id = movie.id
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
        data_json = response.json()
        if 'revenue' in data_json:
            revenue = data_json['revenue']
        else:
            revenue = np.NaN
        return movie_id, revenue
    else:
        return np.NaN, np.NaN

In [9]:
df_2016['id'], df_2016['revenue'] = zip(*df_2016['Title'].apply(lambda x: get_movie_details(str(x))))


C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\3129218827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['id'], df_2016['revenue'] = zip(*df_2016['Title'].apply(lambda x: get_movie_details(str(x))))
C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\3129218827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['id'], df_2016['revenue'] = zip(*df_2016['Title'].apply(lambda x: get_movie_details(str(x))))


In [10]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
        data_json = response.json()
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN  

In [11]:
df_2016['genres'] = df_2016['Title'].map(lambda x: get_genre(str(x)))


C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4282453980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['genres'] = df_2016['Title'].map(lambda x: get_genre(str(x)))


In [12]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [13]:
df_2016['director_name'] = df_2016['Cast and crew'].map(lambda x: get_director(str(x)))


C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\125885639.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2016['director_name'] = df_2016['Cast and crew'].map(lambda x: get_director(str(x)))


In [15]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [16]:
df_2016['actor_1_name'] = df_2016['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [17]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [18]:
df_2016['actor_2_name'] = df_2016['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [19]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [20]:
df_2016['actor_3_name'] = df_2016['Cast and crew'].map(lambda x: get_actor3(str(x)))


In [21]:
df_2016 = df_2016.rename(columns={'Title': 'movie_title'})
df_2016 = df_2016.rename(columns={'id': 'movie_id'})

In [22]:
df_2016 = df_2016.loc[:, ['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'genres', 'movie_title', 'year', 'movie_id', 'revenue']]


In [23]:
# combining the actors names, director, and genre
df_2016['comb'] = df_2016['actor_1_name'] + ' ' + df_2016['actor_2_name'] + ' ' + df_2016['actor_3_name'] + ' ' + df_2016[
    'director_name'] + ' ' + df_2016['genres']

In [24]:
df_2016.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Jason Zada,Natalie Dormer,Taylor Kinney,Yukiyoshi Ozawa,Horror Thriller Mystery,The Forest,2016.0,329440,37608299,Natalie Dormer Taylor Kinney Yukiyoshi Ozawa J...
1,Tim Blake Nelson,Glenn Close,K. Todd Freeman,Jessica Hecht,Drama Thriller,Anesthesia,2016.0,277710,32163,Glenn Close K. Todd Freeman Jessica Hecht Tim ...
2,Ross Partridge,Ross Partridge,Oona Laurence,Jess Weixler,Drama,Lamb,2016.0,324284,0,Ross Partridge Oona Laurence Jess Weixler Ross...
3,Tim Story,Ice Cube,Kevin Hart,Ken Jeong,Action Comedy,Ride Along 2,2016.0,323675,124827316,Ice Cube Kevin Hart Ken Jeong Tim Story Action...
4,Michael Bay,James Badge Dale,John Krasinski,Max Martini,War Action History Drama,13 Hours: The Secret Soldiers of Benghazi,2016.0,300671,69411370,James Badge Dale John Krasinski Max Martini Mi...


## As above create a function to fetch movie details from various years.

In [25]:
def extract_movie_details(wikipedia_link):
    
    source = urllib.request.urlopen(wikipedia_link).read()
    soup = bs.BeautifulSoup(source, 'lxml')
    tables = soup.find_all('table', class_='wikitable sortable')

    # Extracting features from Wikipedia
    df1 = pd.read_html(str(tables[0]))[0]
    df2 = pd.read_html(str(tables[1]))[0]
    df3 = pd.read_html(str(tables[2]))[0]
    df4 = pd.read_html(str(tables[3]))[0]

    df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)
    df_movies = df_movies[['Title', 'Cast and crew']]

    # Add a new column 'year' based on the Wikipedia link
    year = float(wikipedia_link.split("_")[-1])
    df_movies['year'] = year

    # Initialize TMDB API
    tmdb = TMDb()
    tmdb.api_key = 'Your TMDB Key'

    tmdb_movie = Movie()

    def get_movie_details(x):
        result = tmdb_movie.search(x)
        if result:
            movie = result[0]
            movie_id = movie.id
            response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
            data_json = response.json()
            if 'revenue' in data_json:
                revenue = data_json['revenue']
            else:
                revenue = np.NaN
            return movie_id, revenue
        else:
            return np.NaN, np.NaN

    df_movies['id'], df_movies['revenue'] = zip(*df_movies['Title'].apply(lambda x: get_movie_details(str(x))))

    def get_genre(x):
        genres = []
        result = tmdb_movie.search(x)
        if not result:
            return np.NaN
        else:
            movie_id = result[0].id
            response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
            data_json = response.json()
            if data_json['genres']:
                genre_str = " " 
                for i in range(0,len(data_json['genres'])):
                    genres.append(data_json['genres'][i]['name'])
                return genre_str.join(genres)
            else:
                return np.NaN  

    df_movies['genres'] = df_movies['Title'].map(lambda x: get_genre(str(x)))

    def get_director(x):
        if " (director)" in x:
            return x.split(" (director)")[0]
        elif " (directors)" in x:
            return x.split(" (directors)")[0]
        else:
            return x.split(" (director/screenplay)")[0]

    df_movies['director_name'] = df_movies['Cast and crew'].map(lambda x: get_director(str(x)))

    def get_actor1(x):
        return ((x.split("screenplay); ")[-1]).split(", ")[0])

    df_movies['actor_1_name'] = df_movies['Cast and crew'].map(lambda x: get_actor1(str(x)))

    def get_actor2(x):
        if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
            return np.NaN
        else:
            return ((x.split("screenplay); ")[-1]).split(", ")[1])

    df_movies['actor_2_name'] = df_movies['Cast and crew'].map(lambda x: get_actor2(str(x)))

    def get_actor3(x):
        if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
            return np.NaN
        else:
            return ((x.split("screenplay); ")[-1]).split(", ")[2])

    df_movies['actor_3_name'] = df_movies['Cast and crew'].map(lambda x: get_actor3(str(x)))

    df_movies = df_movies.rename(columns={'Title': 'movie_title'})
    df_movies = df_movies.rename(columns={'id': 'movie_id'})
  

    new_df = df_movies.loc[:, ['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'genres', 'movie_title', 'year', 'movie_id', 'revenue']]

    # combining the actors names, director, and genre
    new_df['comb'] = new_df['actor_1_name'] + ' ' + new_df['actor_2_name'] + ' ' + new_df['actor_3_name'] + ' ' + new_df[
        'director_name'] + ' ' + new_df['genres']

    return new_df


In [26]:
link_2017 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2017"
link_2018 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
link_2019 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
link_2020 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
link_2021 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
link_2022 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"
link_2023 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"

In [27]:
df_2017 = extract_movie_details(link_2017)
df_2017.head()

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4235628462.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Anna Foerster,Kate Beckinsale,Theo James,Lara Pulver,Fantasy Action Thriller Horror,Underworld: Blood Wars,2017.0,346672,81093313,Kate Beckinsale Theo James Lara Pulver Anna Fo...
1,Steven C. Miller,Adrian Grenier,John Cusack,Nicolas Cage,Thriller Crime,Arsenal,2017.0,388202,0,Adrian Grenier John Cusack Nicolas Cage Steven...
2,Rafael Palacio Illingworth,Olivia Thirlby,Ben Feldman,Adam Goldberg,Drama,Between Us,2017.0,828373,0,Olivia Thirlby Ben Feldman Adam Goldberg Rafae...
3,Chris Wedge,Lucas Till,Jane Levy,Barry Pepper,Action Comedy Science Fiction,Monster Trucks,2017.0,262841,64493915,Lucas Till Jane Levy Barry Pepper Chris Wedge ...
4,Stacy Title,Douglas Smith,Michael Trucco,Lucien Laviscount,Horror Thriller,The Bye Bye Man,2017.0,292280,26700000,Douglas Smith Michael Trucco Lucien Laviscount...


In [28]:
df_2018 = extract_movie_details(link_2018)
df_2018.head()

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4235628462.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Insidious: The Last Key,2018.0,406563,167885588,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,Christopher Radcliff (director/screenplay); La...,Lauren Wolkstein (director); Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Drama Mystery,The Strange Ones,2018.0,246252,0,Lauren Wolkstein (director); Alex Pettyfer Jam...
2,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller Mystery,The Commuter,2018.0,399035,119942387,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
3,Babak Najafi,Taraji P. Henson,Jahi Di'Allo Winston,Billy Brown,Thriller Action Crime,Proud Mary,2018.0,442064,21753365,Taraji P. Henson Jahi Di'Allo Winston Billy Br...
4,Brett Donowho,Bruce Willis,Cole Hauser,Shawn Ashmore,Action Crime Thriller,Acts of Violence,2018.0,479040,386790,Bruce Willis Cole Hauser Shawn Ashmore Brett D...


In [29]:
df_2019 = extract_movie_details(link_2019)
df_2019.head()

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4235628462.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Horror Thriller Mystery,Escape Room,2019.0,522681,155700000,Taylor Russell Logan Miller Deborah Ann Woll A...
1,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Thriller Drama Action Crime,Rust Creek,2019.0,561362,0,Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Thriller,American Hangman,2019.0,567738,0,Donald Sutherland Vincent Kartheiser Oliver De...
3,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Drama Adventure Family,A Dog's Way Home,2019.0,508763,17643857,Bryce Dallas Howard Edward James Olmos Alexand...
4,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Comedy Drama,The Upside,2019.0,440472,125000000,Bryan Cranston Kevin Hart Nicole Kidman Neil B...


In [30]:
df_2020 = extract_movie_details(link_2020)
df_2020.head()

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4235628462.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge,2020.0,1970.0,183474602.0,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,Underwater,2020.0,443791.0,40882928.0,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss,2020.0,526019.0,26900000.0,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs,2020.0,418533.0,0.0,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper,2020.0,634904.0,0.0,Josh Hartnett Margarita Levieva Chandler Riggs...


In [31]:
df_2021 = extract_movie_details(link_2021)
df_2021.head()

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4235628462.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud,2021.0,675327.0,1054290.0,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Anthony Leone,Amy Cay,Brian Patrick Butler,Michael C. Burgess,Horror,Hacksaw,2021.0,572430.0,0.0,Amy Cay Brian Patrick Butler Michael C. Burges...
2,Yaniv Raz,Lucas Jade Zumann,Taylor Russell,Chase Stokes,Comedy Drama,Dr. Bird's Advice for Sad Poets,2021.0,533705.0,0.0,Lucas Jade Zumann Taylor Russell Chase Stokes ...
3,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,The White Tiger,2021.0,628534.0,0.0,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
4,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Romance,Locked Down,2021.0,741228.0,0.0,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...


In [32]:
df_2022 = extract_movie_details(link_2022)
df_2022.head()

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4235628462.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing,Action Thriller,The 355,2022.0,522016.0,55700000.0,Jessica Chastain Penélope Cruz Fan Bingbing Si...
1,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo,Family Animation Fantasy Horror,The Legend of La Llorona,2022.0,116322.0,3900000.0,Autumn Reeser Danny Trejo Antonio Cupo Patrici...
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando,2022.0,753232.0,0.0,NaN
3,Edward John Drake,Timothy V. Murphy,Bruce Willis,Rob Gough,Action Adventure Thriller,American Siege,2022.0,766907.0,0.0,Timothy V. Murphy Bruce Willis Rob Gough Edwar...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream,2022.0,646385.0,137700000.0,Melissa Barrera Mason Gooding Jenna Ortega Mat...


In [33]:
df_2023 = extract_movie_details(link_2023)
df_2023.head()

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\4235628462.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_movies = df1.append(df2.append(df3.append(df4, ignore_index=True), ignore_index=True), ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald,Science Fiction Horror,M3GAN,2023.0,536554,179960167,Allison Williams Violet McGraw Amie Donald Ger...
1,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN,Western Drama,The Old Way,2023.0,872954,0,NaN
2,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall,Horror Fantasy Science Fiction Thriller,The Devil Conspiracy,2023.0,296271,0,Alice Orr-Ewing Joe Doyle Eveline Hall Nathan ...
3,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An,Action,Plane,2023.0,646389,51000000,Gerard Butler Mike Colter Yoson An Jean-Franço...
4,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom,Comedy,House Party,2023.0,632065,0,Tosin Cole Jacob Latimore Karen Obilom Calmati...


In [34]:
df_combined = df_2016.append([df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023], ignore_index=True)
df_combined

C:\Users\aditya\AppData\Local\Temp\ipykernel_22320\180449076.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combined = df_2016.append([df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023], ignore_index=True)


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Jason Zada,Natalie Dormer,Taylor Kinney,Yukiyoshi Ozawa,Horror Thriller Mystery,The Forest,2016.0,329440.0,37608299.0,Natalie Dormer Taylor Kinney Yukiyoshi Ozawa J...
1,Tim Blake Nelson,Glenn Close,K. Todd Freeman,Jessica Hecht,Drama Thriller,Anesthesia,2016.0,277710.0,32163.0,Glenn Close K. Todd Freeman Jessica Hecht Tim ...
2,Ross Partridge,Ross Partridge,Oona Laurence,Jess Weixler,Drama,Lamb,2016.0,324284.0,0.0,Ross Partridge Oona Laurence Jess Weixler Ross...
3,Tim Story,Ice Cube,Kevin Hart,Ken Jeong,Action Comedy,Ride Along 2,2016.0,323675.0,124827316.0,Ice Cube Kevin Hart Ken Jeong Tim Story Action...
4,Michael Bay,James Badge Dale,John Krasinski,Max Martini,War Action History Drama,13 Hours: The Secret Soldiers of Benghazi,2016.0,300671.0,69411370.0,James Badge Dale John Krasinski Max Martini Mi...
...,...,...,...,...,...,...,...,...,...,...
2195,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,Memory,2023.0,818397.0,13900000.0,Jessica Chastain Peter Sarsgaard Merritt Wever...
2196,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,Drama,The Color Purple,2023.0,558915.0,66025125.0,Fantasia Barrino Taraji P. Henson Danielle Bro...
2197,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,The Boys in the Boat,2023.0,823452.0,54401583.0,Callum Turner Joel Edgerton Peter Guinness Geo...
2198,Michael Mann,Adam Driver,Penelope Cruz,Shailene Woodley,History Drama,Ferrari,2023.0,365620.0,39220516.0,Adam Driver Penelope Cruz Shailene Woodley Mic...


In [35]:
df_combined['genres'].unique()

array(['Horror Thriller Mystery', 'Drama Thriller', 'Drama',
       'Action Comedy', 'War Action History Drama',
       'Adventure Animation Comedy Family', 'Comedy',
       'Science Fiction Adventure Action',
       'Thriller Mystery Science Fiction',
       'Action Adventure Animation Comedy Family',
       'Action Drama History Thriller', 'Romance Comedy',
       'Western Action Drama', 'Comedy Crime Mystery',
       'Romance Horror Comedy Thriller', 'Drama Romance',
       'Action Adventure Comedy', 'Comedy Adventure Action',
       'Comedy Romance', 'Action Adventure Drama Fantasy',
       'Horror Mystery Drama', 'Action Adventure Drama',
       'Action Adventure Fantasy', 'Action Crime Adventure Thriller',
       'Comedy Drama History Adventure',
       'Animation Adventure Family Comedy', 'Action Thriller',
       'Comedy Drama War', 'Thriller Horror',
       'Animation Comedy Family Adventure',
       'Thriller Science Fiction Drama Horror', 'Comedy Drama Romance',
       'Fami

In [36]:
# change the "Science Fiction" genre name to 'Sci-Fi'
df_combined['genres'] = df_combined['genres'].str.replace('Science Fiction', 'Sci-Fi')

In [37]:
df_combined['genres'].unique()

array(['Horror Thriller Mystery', 'Drama Thriller', 'Drama',
       'Action Comedy', 'War Action History Drama',
       'Adventure Animation Comedy Family', 'Comedy',
       'Sci-Fi Adventure Action', 'Thriller Mystery Sci-Fi',
       'Action Adventure Animation Comedy Family',
       'Action Drama History Thriller', 'Romance Comedy',
       'Western Action Drama', 'Comedy Crime Mystery',
       'Romance Horror Comedy Thriller', 'Drama Romance',
       'Action Adventure Comedy', 'Comedy Adventure Action',
       'Comedy Romance', 'Action Adventure Drama Fantasy',
       'Horror Mystery Drama', 'Action Adventure Drama',
       'Action Adventure Fantasy', 'Action Crime Adventure Thriller',
       'Comedy Drama History Adventure',
       'Animation Adventure Family Comedy', 'Action Thriller',
       'Comedy Drama War', 'Thriller Horror',
       'Animation Comedy Family Adventure',
       'Thriller Sci-Fi Drama Horror', 'Comedy Drama Romance',
       'Family Drama', 'Adventure Sci-Fi Actio

In [38]:
df_combined.to_csv('movies_3.csv',index=False)

In [39]:
df_combined

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,year,movie_id,revenue,comb
0,Jason Zada,Natalie Dormer,Taylor Kinney,Yukiyoshi Ozawa,Horror Thriller Mystery,The Forest,2016.0,329440.0,37608299.0,Natalie Dormer Taylor Kinney Yukiyoshi Ozawa J...
1,Tim Blake Nelson,Glenn Close,K. Todd Freeman,Jessica Hecht,Drama Thriller,Anesthesia,2016.0,277710.0,32163.0,Glenn Close K. Todd Freeman Jessica Hecht Tim ...
2,Ross Partridge,Ross Partridge,Oona Laurence,Jess Weixler,Drama,Lamb,2016.0,324284.0,0.0,Ross Partridge Oona Laurence Jess Weixler Ross...
3,Tim Story,Ice Cube,Kevin Hart,Ken Jeong,Action Comedy,Ride Along 2,2016.0,323675.0,124827316.0,Ice Cube Kevin Hart Ken Jeong Tim Story Action...
4,Michael Bay,James Badge Dale,John Krasinski,Max Martini,War Action History Drama,13 Hours: The Secret Soldiers of Benghazi,2016.0,300671.0,69411370.0,James Badge Dale John Krasinski Max Martini Mi...
...,...,...,...,...,...,...,...,...,...,...
2195,Michel Franco,Jessica Chastain,Peter Sarsgaard,Merritt Wever,Action Thriller Crime,Memory,2023.0,818397.0,13900000.0,Jessica Chastain Peter Sarsgaard Merritt Wever...
2196,Blitz Bazawule,Fantasia Barrino,Taraji P. Henson,Danielle Brooks,Drama,The Color Purple,2023.0,558915.0,66025125.0,Fantasia Barrino Taraji P. Henson Danielle Bro...
2197,George Clooney,Callum Turner,Joel Edgerton,Peter Guinness,Drama History,The Boys in the Boat,2023.0,823452.0,54401583.0,Callum Turner Joel Edgerton Peter Guinness Geo...
2198,Michael Mann,Adam Driver,Penelope Cruz,Shailene Woodley,History Drama,Ferrari,2023.0,365620.0,39220516.0,Adam Driver Penelope Cruz Shailene Woodley Mic...
